# Importazione Librerie

In [ ]:
%pip install numpy==1.24.4

%pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html

%pip install mmcv==2.0.1

%pip install mmdet==3.0.0

%rm -rf mmpose
!git clone https://github.com/open-mmlab/mmpose.git
%cd mmpose

%pip install -r requirements.txt

%pip install -e .


# Caricamento cartella con i video

In [ ]:
!apt-get install unrar -y
!pip install patool

In [5]:
import os
from google.colab import files
import patoolib


uploaded = files.upload()

# nome del file
rar_filename = next(iter(uploaded))
print(f"\nArchivio caricato: {rar_filename}")

# cartella di output
output_dir = "/content/estratti_video"
os.makedirs(output_dir, exist_ok=True)

# estrazione
patoolib.extract_archive(rar_filename, outdir=output_dir)

print("\nEstrazione completata.")

INFO patool: Extracting 12_10_2023.rar ...
INFO:patool:Extracting 12_10_2023.rar ...
INFO patool: running /usr/bin/unrar x -kb -or -- /content/12_10_2023.rar
INFO:patool:running /usr/bin/unrar x -kb -or -- /content/12_10_2023.rar
INFO patool: ... 12_10_2023.rar extracted to `/content/estratti_video'.
INFO:patool:... 12_10_2023.rar extracted to `/content/estratti_video'.


Saving 12_10_2023.rar to 12_10_2023.rar

Archivio caricato: 12_10_2023.rar

Estrazione completata.


# Crop

In [2]:
giorno_cartella = "12_10_2023"

In [7]:
import os
import cv2

def crop_video_rect(input_path, output_path, crop_x, crop_y, crop_width, crop_height):
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print(f"Errore nell'aprire il video: {input_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (crop_width, crop_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cropped = frame[crop_y:crop_y+crop_height, crop_x:crop_x+crop_width]

        if cropped.shape[0] != crop_height or cropped.shape[1] != crop_width:
            print(f"Frame fuori dai limiti: {input_path}. Skipping frame.")
            continue

        out.write(cropped)

    cap.release()
    out.release()
    print(f"Salvato: {output_path}")
    files.download(output_path)


def process_all_videos(input_dir, output_dir, crop_x, crop_y, crop_width, crop_height):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith('.mp4'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            print(f"\nCropping: {filename}")
            crop_video_rect(input_path, output_path, crop_x, crop_y, crop_width, crop_height)



input_dir = f"/content/estratti_video/{giorno_cartella}"
output_dir = f"/content/cropped_videos/{giorno_cartella}"

crop_x = 0
crop_y = 150
crop_width = 720
crop_height = 710


process_all_videos(input_dir, output_dir, crop_x, crop_y, crop_width, crop_height)



Cropping: 75.mp4
Salvato: /content/cropped_videos/12_10_2023/75.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Cropping: 72.mp4
Salvato: /content/cropped_videos/12_10_2023/72.mp4


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Cropping: 74.mp4
Salvato: /content/cropped_videos/12_10_2023/74.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Keypoints extraction

## Importazione checkpoint RTMPose-x

“We employ the RTMPose-x from MMPose to extract whole-body keypoints”

In [8]:
!mkdir -p checkpoints

!wget https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-x_simcc-coco-wholebody_pt-body7_270e-384x288-401dfc90_20230629.pth  \
      -O /content/mmpose/checkpoints/rtmpose-x.pth

!wget https://huggingface.co/facebook/sapiens-pose-bbox-detector/resolve/main/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth \
     -O /content/mmpose/checkpoints/rtmdet.pth

--2025-06-02 17:46:09--  https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-x_simcc-coco-wholebody_pt-body7_270e-384x288-401dfc90_20230629.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.23.184, 47.246.23.182, 47.246.23.180, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.23.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 227232559 (217M) [application/octet-stream]
Saving to: ‘/content/mmpose/checkpoints/rtmpose-x.pth’

/content/mmpose/che 100%[===================>] 216.71M  17.6MB/s    in 13s     

2025-06-02 17:46:23 (16.9 MB/s) - ‘/content/mmpose/checkpoints/rtmpose-x.pth’ saved [227232559/227232559]

--2025-06-02 17:46:23--  https://huggingface.co/facebook/sapiens-pose-bbox-detector/resolve/main/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.114, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163

## Estrazione

In [ ]:

# DOCUMENTAZIONE:  https://mmpose.readthedocs.io/en/latest/overview.html

"""
python demo/topdown_demo_with_mmdet.py \
  <det-config> <det-checkpoint> \
  <pose-config> <pose-checkpoint> \
  --input <input_path> \
  --output-root <output_path> \
  [--device <device>] \
  [--save-predictions] \
  [--draw-heatmap] \
  [--show]
"""

"""
python demo/topdown_demo_with_mmdet.py \
    demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py \
    https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth \
    configs/wholebody_2d_keypoint/topdown_heatmap/coco-wholebody/td-hm_hrnet-w48_dark-8xb32-210e_coco-wholebody-384x288.py \
    https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth \
    --input https://user-images.githubusercontent.com/87690686/137440639-fb08603d-9a35-474e-b65f-46b5c06b68d6.mp4 \
    --output-root vis_results/ --show
"""

In [15]:
import os
import glob


video_folder = f"/content/cropped_videos/{giorno_cartella}"
output_base = f"/content/output_video/{giorno_cartella}"

mmdet_cfg = "/content/mmpose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py"
mmdet_ckpt = "/content/mmpose/checkpoints/rtmdet.pth"
pose_cfg = "/content/mmpose/configs/wholebody_2d_keypoint/rtmpose/coco-wholebody/rtmpose-x_8xb32-270e_coco-wholebody-384x288.py"
pose_ckpt = "/content/mmpose/checkpoints/rtmpose-x.pth"

device = "cpu"

# trova tutti i file .mp4 in tutte le sottocartelle
video_files = glob.glob(os.path.join(video_folder, '**', '*.mp4'), recursive=True)

# Ccrea cartella output
os.makedirs(output_base, exist_ok=True)


for video_path in video_files:

    # estrai percorso relativo rispetto a video_folder
    rel_path = os.path.relpath(video_path, video_folder)
    base_name = os.path.splitext(rel_path.replace(os.sep, "_"))[0]
    output_dir = os.path.join(output_base, base_name)
    os.makedirs(output_dir, exist_ok=True)

    print(f"\nElaborazione di: {video_path} -> {output_dir}\n")

    # comando estrazione keypoints
    !python /content/mmpose/demo/topdown_demo_with_mmdet.py \
        {mmdet_cfg} \
        {mmdet_ckpt} \
        {pose_cfg} \
        {pose_ckpt} \
        --input "{video_path}" \
        --output-root "{output_dir}" \
        --save-predictions \
        --device {device}

print("\n\nTutti i video sono stati elaborati.")


Elaborazione di: /content/cropped_videos/12_10_2023/75.mp4 -> /content/output_video/12_10_2023/75

Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.1+cu118
/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Loads checkpoint by local backend from path: /content/mmpose/checkpoints/rtmdet.pth
Loads checkpoint by local backend from path: /content/mmpose/checkpoints/rtmpose-x.pth
/usr/local/lib/python3.11/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/pytho

# Adattamento output in JSON e salvataggio in PKL

In [16]:
import json
import pickle
import numpy as np
import os
from glob import glob


def json_to_pkl(json_path, pkl_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open(pkl_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"JSON convertito in PKL: {pkl_path}")


def process_and_convert(json_path):

    # estrae il nome
    file_id = os.path.splitext(os.path.basename(json_path))[0].replace("results_", "")

    print(f"\nProcessamento : {json_path}")

    # JSON originale
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # nuova struttura corretta
    output_corrected = {
        "keypoints": [],
        "scores": [],
        "w_h": [720, 710]   #  850, 720

    }

    for frame in data["instance_info"]:
        frame_kps = []
        frame_scores = []
        for instance in frame["instances"]:
            kps = instance["keypoints"]
            scores = instance["keypoint_scores"]
            frame_kps.append([[x, y] for x, y in kps])
            frame_scores.append(scores)
        output_corrected["keypoints"].append(frame_kps)
        output_corrected["scores"].append(frame_scores)

    # salvataggio JSON corretto
    corrected_json_path = os.path.join(os.path.dirname(json_path), f"{file_id}.json")
    with open(corrected_json_path, "w", encoding="utf-8") as f:
        json.dump(output_corrected, f, indent=4)
    print(f"Salvato JSON corretto: {corrected_json_path}")

    # conversione in PKL
    #pkl_output_path = os.path.join(os.path.dirname(json_path), f"{file_id}.pkl")
    pkl_output_path = os.path.join(os.path.dirname(json_path), f"{giorno_cartella}_{file_id}.pkl")
    json_to_pkl(corrected_json_path, pkl_output_path)
    print(f"Salvato pkl : {pkl_output_path}")
    files.download(pkl_output_path)



base_dir = f"/content/output_video/{giorno_cartella}"
json_files = glob(os.path.join(base_dir, "**", "results*.json"), recursive=True)

print(f"Trovati {len(json_files)} file .json da processare...\n")

for json_file in json_files:
    try:
        process_and_convert(json_file)
    except Exception as e:
        print(f"\nErrore su {json_file}: {e}")


Trovati 10 file .json da processare...


Processamento : /content/output_video/12_10_2023/76/results_76.json
Salvato JSON corretto: /content/output_video/12_10_2023/76/76.json
JSON convertito in PKL: /content/output_video/12_10_2023/76/12_10_2023_76.pkl
Salvato pkl : /content/output_video/12_10_2023/76/12_10_2023_76.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Processamento : /content/output_video/12_10_2023/74/results_74.json
Salvato JSON corretto: /content/output_video/12_10_2023/74/74.json
JSON convertito in PKL: /content/output_video/12_10_2023/74/12_10_2023_74.pkl
Salvato pkl : /content/output_video/12_10_2023/74/12_10_2023_74.pkl


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Processamento : /content/output_video/12_10_2023/73/results_73.json
Salvato JSON corretto: /content/output_video/12_10_2023/73/73.json
JSON convertito in PKL: /content/output_video/12_10_2023/73/12_10_2023_73.pkl
Salvato pkl : /content/output_video/12_10_2023/73/12_10_2023_73.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Allineamento con annotazioni

## Caricamento file txt annotazioni

In [17]:
import os
from google.colab import files

%cd ..
uploaded = files.upload()


/content


Saving 12_10_2023.txt to 12_10_2023.txt


In [18]:
import os
import json
from glob import glob


base_dir = f"/content/output_video/{giorno_cartella}"
text_file = f"/content/{giorno_cartella}.txt"

output_json_path = f"/content/LIS_Labels_{giorno_cartella}.json"

# carica tutte le righe del file .txt
with open(text_file, "r", encoding="utf-8") as f:
    text_lines = [line.strip().split("--", 2)[-1].strip() for line in f]


pkl_files = sorted(glob(os.path.join(base_dir, "**", "*.pkl"), recursive=True))


result = []

for pkl_path in pkl_files:
    filename = os.path.basename(pkl_path)
    #file_id = os.path.splitext(filename)[0]
    file_id = os.path.splitext(filename)[0].replace(f"{giorno_cartella}_", "")

    video_filename = f"{giorno_cartella}_{file_id}.mp4"
    pose_filename = f"{giorno_cartella}_{file_id}.pkl"

    print(f"Processing file: {pkl_path}, ID: {file_id}")
    # recupera testo dalla riga corrispondente all'indice
    try:
        text = text_lines[int(file_id)]
    except (IndexError, ValueError):
        print(f"Errore con ID {file_id}, salta...")
        continue

    entry = {
        "video": video_filename,
        "pose": pose_filename,
        "text": text
    }

    result.append(entry)
    print(f"\nAggiunto {giorno_cartella}_{file_id} al JSON")

# JSON finale
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

print(f"\nCreato file JSON con {len(result)} elementi: {output_json_path}")

files.download(output_json_path)


Processing file: /content/output_video/12_10_2023/71/12_10_2023_71.pkl, ID: 71

Aggiunto 12_10_2023_71 al JSON
Processing file: /content/output_video/12_10_2023/72/12_10_2023_72.pkl, ID: 72

Aggiunto 12_10_2023_72 al JSON
Processing file: /content/output_video/12_10_2023/73/12_10_2023_73.pkl, ID: 73

Aggiunto 12_10_2023_73 al JSON
Processing file: /content/output_video/12_10_2023/74/12_10_2023_74.pkl, ID: 74

Aggiunto 12_10_2023_74 al JSON
Processing file: /content/output_video/12_10_2023/75/12_10_2023_75.pkl, ID: 75

Aggiunto 12_10_2023_75 al JSON
Processing file: /content/output_video/12_10_2023/76/12_10_2023_76.pkl, ID: 76

Aggiunto 12_10_2023_76 al JSON
Processing file: /content/output_video/12_10_2023/77/12_10_2023_77.pkl, ID: 77

Aggiunto 12_10_2023_77 al JSON
Processing file: /content/output_video/12_10_2023/78/12_10_2023_78.pkl, ID: 78

Aggiunto 12_10_2023_78 al JSON
Processing file: /content/output_video/12_10_2023/79/12_10_2023_79.pkl, ID: 79

Aggiunto 12_10_2023_79 al JSON
P

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
import shutil
shutil.rmtree("/content/estratti_video")
shutil.rmtree("/content/cropped_videos")
shutil.rmtree("/content/output_video")